# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import math
import requests
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
df = pd.read_csv('sp_500_stocks.csv')

with open('token_key.csv', 'r') as f:
    IEX_CLOUD_API_TOKEN = f.read()
    
print(df.head(), IEX_CLOUD_API_TOKEN)

  Ticker
0      A
1    AAL
2    AAP
3   AAPL
4   ABBV Tpk_059b97af715d417d9f49f50b51b1c448


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [8]:
# check the iex cloud api and find key stats. copy the GET request url
# find sandbox too and add before the key stats. Add stable after the sandbox url

symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
api_data = requests.get(api_url).json()
#print(api_data.status_code)
print(api_data)

{'companyName': 'Apple Inc', 'marketcap': 2181334532577, 'week52high': 139, 'week52low': 56.09, 'week52change': 0.667783254448259, 'sharesOutstanding': 17231040823, 'float': 0, 'avg10Volume': 108006945, 'avg30Volume': 114944768, 'day200MovingAvg': 116.91, 'day50MovingAvg': 130.23, 'employees': 0, 'ttmEPS': 3.38, 'ttmDividendRate': 0.8112142616887611, 'dividendYield': 0.00648305802337445, 'nextDividendDate': '0', 'exDividendDate': '2020-11-02', 'nextEarningsDate': '0', 'peRatio': 38.91144683652143, 'beta': 1.1896123421097022, 'maxChangePercent': 48.54584411635317, 'year5ChangePercent': 4.59274080423026, 'year2ChangePercent': 2.61946118557047, 'year1ChangePercent': 0.6863375560730219, 'ytdChangePercent': -0.02936615515871044, 'month6ChangePercent': 0.3487080389796292, 'month3ChangePercent': 0.06674761816692726, 'month1ChangePercent': 0.06093229077597714, 'day30ChangePercent': 0.008203103323095397, 'day5ChangePercent': -0.016013964750590103}


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [10]:
api_data['year1ChangePercent']

0.6863375560730219

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [13]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(df['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol in symbol_strings:
    print(symbol)
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [46]:
df2 = pd.DataFrame(columns = my_columns)

for symbols in symbol_strings[:1]:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=stats,quote&token={IEX_CLOUD_API_TOKEN}' 
    data = requests.get(batch_api_url_call).json()
    #print(symbols.split(','))
    #print(data['AAPL']['price'])
    for symbol in symbols.split(','):
        pd_series = pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A'
                    ], index = my_columns)

        df2 = df2.append(pd_series, ignore_index = True)
    
        
df2.head(20)

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,128.99,0.457079,N/A
1,AAL,16.77,-0.411360,N/A
2,AAP,175.07,0.147411,N/A
3,AAPL,129.79,0.684248,N/A
4,ABBV,113.75,0.347993,N/A
5,ABC,106.05,0.213199,N/A
6,ABMD,328.58,0.866043,N/A
7,ABT,114.85,0.314668,N/A
8,ACN,259.00,0.254369,N/A
9,ADBE,470.09,0.353501,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [60]:
df2.sort_values('One-Year Price Return', ascending = False, inplace=True)
# for simplicity we work with 50 data
df3 = df2.head(50).reset_index()
df3 = df3.drop('index', axis= 'columns')
df3

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,42.40,2.500000,N/A
1,ALB,187.97,1.402375,N/A
2,ALGN,562.12,0.965524,N/A
3,AMD,91.56,0.902727,N/A
4,CDNS,140.80,0.878585,N/A
5,ABMD,328.58,0.866043,N/A
6,AMAT,108.00,0.725241,N/A
7,AAPL,129.79,0.684248,N/A
8,AMZN,3237.50,0.678607,N/A
9,ADSK,323.58,0.643426,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [57]:
def Number_of_shares_to_buy(portfolio):
    #my_portfolio = input('Enter the value of your portfolio: ')
    if type(portfolio) == str:
        return "Value must be a Number e.g 10000"
    else:
        val = float(portfolio) / len(df3)
        position_size = math.floor(val)
        
    return position_size

Number_of_shares_to_buy('My portfolio size is too small')

'Value must be a Number e.g 10000'

In [67]:
for i in range(0, len(df3)):
    position_size = Number_of_shares_to_buy(1000000)
    df3.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / df3.loc[i, 'Price'])

df3

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,42.40,2.500000,471
1,ALB,187.97,1.402375,106
2,ALGN,562.12,0.965524,35
3,AMD,91.56,0.902727,218
4,CDNS,140.80,0.878585,142
5,ABMD,328.58,0.866043,60
6,AMAT,108.00,0.725241,185
7,AAPL,129.79,0.684248,154
8,AMZN,3237.50,0.678607,6
9,ADSK,323.58,0.643426,61


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [114]:
hqm_columns = [
    'ticker',
    'price',
    'number of shares to buy',
    'one year price return',
    'one year return percentile',
    'six month price return',
    'six month return percentile',
    'three month price return',
    'three month return percentile',
    'one month price return',
    'one month return percentile',
    'hqm value'
]

hqm_df = pd.DataFrame(columns = hqm_columns)
#hqm_df

for symbols in symbol_strings[:1]:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}' 
    data = requests.get(batch_api_url_call).json()
    
    for symbol in symbols.split(','):
        pd_series = pd.Series(
        [
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'  
        ], index = hqm_columns
        )
        hqm_df = hqm_df.append(pd_series, ignore_index = True)
        
hqm_df

,ticker,price,number of shares to buy,one year price return,one year return percentile,six month price return,six month return percentile,three month price return,three month return percentile,one month price return,one month return percentile,hqm value
0,A,131.22,N/A,0.426741,N/A,0.369035,N/A,0.190805,N/A,0.079914,N/A,N/A
1,AAL,16.26,N/A,-0.449246,N/A,0.278530,N/A,0.276072,N/A,-0.066242,N/A,N/A
2,AAP,167.56,N/A,0.098496,N/A,0.143957,N/A,0.069166,N/A,0.026952,N/A,N/A
3,AAPL,131.39,N/A,0.634282,N/A,0.329927,N/A,0.070254,N/A,-0.005381,N/A,N/A
4,ABBV,114.90,N/A,0.319306,N/A,0.135858,N/A,0.303425,N/A,0.069363,N/A,N/A
5,ABC,111.30,N/A,0.194271,N/A,0.048573,N/A,0.081221,N/A,0.088989,N/A,N/A
6,ABMD,332.57,N/A,0.831449,N/A,0.211603,N/A,0.155533,N/A,0.206365,N/A,N/A
7,ABT,111.60,N/A,0.289880,N/A,0.168264,N/A,0.019465,N/A,0.042267,N/A,N/A
8,ACN,266.02,N/A,0.242099,N/A,0.173916,N/A,0.106749,N/A,0.028899,N/A,N/A
9,ADBE,468.49,N/A,0.331588,N/A,0.075910,N/A,-0.089259,N/A,-0.065324,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [115]:


time_periods = [
    'one year',
    'six month',
    'three month',
    'one month'
]
for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} return percentile'] = \
            stats.percentileofscore(hqm_df[f'{time_period} price return' ], \
                                   hqm_df.loc[row, f'{time_period} price return'])
        
hqm_df.head()

,ticker,price,number of shares to buy,one year price return,one year return percentile,six month price return,six month return percentile,three month price return,three month return percentile,one month price return,one month return percentile,hqm value
0,A,131.22,N/A,0.426741,85,0.369035,78,0.190805,67,0.079914,76,N/A
1,AAL,16.26,N/A,-0.449246,3,0.278530,61,0.276072,79,-0.066242,6,N/A
2,AAP,167.56,N/A,0.098496,52,0.143957,39,0.069166,36,0.026952,39,N/A
3,AAPL,131.39,N/A,0.634282,92,0.329927,75,0.070254,37,-0.005381,25,N/A
4,ABBV,114.90,N/A,0.319306,75,0.135858,37,0.303425,82,0.069363,69,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [134]:
(76+85+78+67)/4

76.5

In [133]:
# note that the HQM Score is the same as the hqm_value
for row in hqm_df.index:
    momentum_percentile_sum = []
    for time_period in time_periods:
        momentum_percentile_sum.append(hqm_df.loc[row, [f'{time_period} return percentile']])
    hqm_df.loc[row, 'hqm value' ] = np.mean(momentum_percentile_sum)
    
hqm_df.drop('hqm_value', axis=1, inplace=True)
hqm_df.head()

,ticker,price,number of shares to buy,one year price return,one year return percentile,six month price return,six month return percentile,three month price return,three month return percentile,one month price return,one month return percentile,hqm value
0,A,131.22,N/A,0.426741,85,0.369035,78,0.190805,67,0.079914,76,76
1,AAL,16.26,N/A,-0.449246,3,0.278530,61,0.276072,79,-0.066242,6,37
2,AAP,167.56,N/A,0.098496,52,0.143957,39,0.069166,36,0.026952,39,41
3,AAPL,131.39,N/A,0.634282,92,0.329927,75,0.070254,37,-0.005381,25,57
4,ABBV,114.90,N/A,0.319306,75,0.135858,37,0.303425,82,0.069363,69,65


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [143]:
hqm_df = hqm_df.sort_values('hqm value', ascending=False)[:51]
hqm_df = hqm_df.reset_index()

,level_0,index,ticker,price,number of shares to buy,one year price return,one year return percentile,six month price return,six month return percentile,three month price return,three month return percentile,one month price return,one month return percentile,hqm value
0,0,23,ALB,186.03,N/A,1.327218,99,1.133353,100,0.956911,100,0.334047,100,99
1,2,24,ALGN,572.69,N/A,1.024170,98,0.845025,98,0.701202,97,0.101880,85,94
2,1,29,AMAT,106.54,N/A,0.661613,93,0.671313,95,0.673679,96,0.172133,95,94
3,3,16,AES,28.80,N/A,0.399419,82,0.912846,99,0.416816,89,0.263258,99,92
4,4,45,APTV,149.89,N/A,0.592048,89,0.743277,96,0.476119,90,0.169978,94,92
5,5,37,ANET,326.88,N/A,0.449822,86,0.479119,88,0.409768,88,0.111545,88,87
6,6,78,CARR,41.21,N/A,2.405000,100,0.536040,90,0.246996,75,0.068486,68,83
7,7,90,CFG,41.42,N/A,0.090622,50,0.648949,94,0.557135,93,0.150289,92,82
8,8,10,ADI,163.84,N/A,0.359416,79,0.374073,79,0.294855,81,0.107047,87,81
9,9,98,CMA,65.25,N/A,-0.001076,29,0.822829,97,0.557420,94,0.204398,96,79


In [148]:
hqm_df.drop(['level_0','index'], axis=1, inplace=True)
hqm_df.head()

,ticker,price,number of shares to buy,one year price return,one year return percentile,six month price return,six month return percentile,three month price return,three month return percentile,one month price return,one month return percentile,hqm value
0,ALB,186.03,N/A,1.327218,99,1.133353,100,0.956911,100,0.334047,100,99
1,ALGN,572.69,N/A,1.024170,98,0.845025,98,0.701202,97,0.101880,85,94
2,AMAT,106.54,N/A,0.661613,93,0.671313,95,0.673679,96,0.172133,95,94
3,AES,28.80,N/A,0.399419,82,0.912846,99,0.416816,89,0.263258,99,92
4,APTV,149.89,N/A,0.592048,89,0.743277,96,0.476119,90,0.169978,94,92


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [151]:
for i in range(0, len(hqm_df)):
    position_size = Number_of_shares_to_buy(1000000)
    hqm_df.loc[i, 'number of shares to buy'] = math.floor(position_size / hqm_df.loc[i, 'price'])

hqm_df

,ticker,price,number of shares to buy,one year price return,one year return percentile,six month price return,six month return percentile,three month price return,three month return percentile,one month price return,one month return percentile,hqm value
0,ALB,186.03,5375,1.327218,99,1.133353,100,0.956911,100,0.334047,100,99
1,ALGN,572.69,1746,1.024170,98,0.845025,98,0.701202,97,0.101880,85,94
2,AMAT,106.54,9386,0.661613,93,0.671313,95,0.673679,96,0.172133,95,94
3,AES,28.80,34722,0.399419,82,0.912846,99,0.416816,89,0.263258,99,92
4,APTV,149.89,6671,0.592048,89,0.743277,96,0.476119,90,0.169978,94,92
5,ANET,326.88,3059,0.449822,86,0.479119,88,0.409768,88,0.111545,88,87
6,CARR,41.21,24265,2.405000,100,0.536040,90,0.246996,75,0.068486,68,83
7,CFG,41.42,24142,0.090622,50,0.648949,94,0.557135,93,0.150289,92,82
8,ADI,163.84,6103,0.359416,79,0.374073,79,0.294855,81,0.107047,87,81
9,CMA,65.25,15325,-0.001076,29,0.822829,97,0.557420,94,0.204398,96,79


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: